In [1]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


--2024-09-02 02:27:27--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.10, 13.227.219.70, 13.227.219.59, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   132MB/s    in 21s     

2024-09-02 02:27:48 (118 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [2]:
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'


  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-gds1fk27
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-gds1fk27
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=6cd48c6872740a4bd6a97cb4302ee190f9fdae23783df70c113dcb23acf2e2ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-hif5d3pj/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment_anything


# Automatically generating object masks with SAM

Since SAM can efficiently process prompts, masks for the entire image can be generated by sampling a large number of prompts over an image. This method was used to generate the dataset SA-1B.

The class `SamAutomaticMaskGenerator` implements this capability. It works by sampling single-point input prompts in a grid over the image, from each of which SAM can predict multiple masks. Then, masks are filtered for quality and deduplicated using non-maximal suppression. Additional options allow for further improvement of mask quality and quantity, such as running prediction on multiple crops of the image or postprocessing masks to remove small disconnected regions and holes.

In [ ]:
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/github/facebookresearch/segment-anything/blob/main/notebooks/automatic_mask_generator_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

## Environment Set-up

If running locally using jupyter, first install `segment_anything` in your environment using the [installation instructions](https://github.com/facebookresearch/segment-anything#installation) in the repository. If running from Google Colab, set `using_colab=True` below and run the cell. In Colab, be sure to select 'GPU' under 'Edit'->'Notebook Settings'->'Hardware accelerator'.

In [3]:
using_colab = True

In [4]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.4.0+cu121
Torchvision version: 0.19.0+cu121
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-i3czloq0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-i3czloq0
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
--2024-09-02 02:30:27--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99846 (98K) [image/jpeg]
Saving to: ‘images/dog.jpg’

dog.jpg             100%[===================>]  97.51K  --

## Set-up

In [5]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [6]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

## Example image

In [7]:
image = cv2.imread('/content/sample_data/naver_map3.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [8]:
plt.figure(figsize=(20,20))
plt.imshow(image)
plt.axis('off')
plt.show()

Output hidden; open in https://colab.research.google.com to view.

## Automatic mask generation

To run automatic mask generation, provide a SAM model to the `SamAutomaticMaskGenerator` class. Set the path below to the SAM checkpoint. Running on CUDA and with the default model is recommended.

In [9]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


To generate masks, just run `generate` on an image.

In [10]:
masks = mask_generator.generate(image)

Mask generation returns a list over masks, where each mask is a dictionary containing various data about the mask. These keys are:
* `segmentation` : the mask
* `area` : the area of the mask in pixels
* `bbox` : the boundary box of the mask in XYWH format
* `predicted_iou` : the model's own prediction for the quality of the mask
* `point_coords` : the sampled input point that generated this mask
* `stability_score` : an additional measure of mask quality
* `crop_box` : the crop of the image used to generate this mask in XYWH format

In [11]:
print(len(masks))
print(masks[0].keys())

111
dict_keys(['segmentation', 'area', 'bbox', 'predicted_iou', 'point_coords', 'stability_score', 'crop_box'])


Show all the masks overlayed on the image.

In [12]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [13]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests  # 여기에 requests 모듈 임포트 추가
import json

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    # 마스크된 객체 영역만 잘라내기
    masked_image = Image.fromarray(image * mask[..., np.newaxis])

    # 전처리 후 모델에 입력
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    # 분류 결과 얻기
    _, predicted_class = output.max(1)

    return predicted_class.item()

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

for mask in masks:
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    class_id = classify_masked_region(image_np, mask_np)
    label = imagenet_labels[class_id]

    # 객체 경계 그리기 및 텍스트 레이블 추가
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), label, color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

plt.show()


/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


KeyboardInterrupt: 

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# 임시 클래스 매핑
custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    # 필요한 경우 다른 레이블도 매핑
}

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    # 마스크된 객체 영역만 잘라내기
    masked_image = Image.fromarray(image * mask[..., np.newaxis])

    # 전처리 후 모델에 입력
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    # 분류 결과 얻기
    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 커스텀 라벨로 매핑
    return custom_labels.get(original_label, original_label)  # 매핑된 값이 없으면 원래 레이블 반환

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

for mask in masks:
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 객체 경계 그리기 및 텍스트 레이블 추가
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), label, color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# 임시 클래스 매핑
custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    # 필요한 경우 다른 레이블도 매핑
}

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    # 마스크된 객체 영역만 잘라내기
    masked_image = Image.fromarray(image * mask[..., np.newaxis])

    # 전처리 후 모델에 입력
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    # 분류 결과 얻기
    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 커스텀 라벨로 매핑
    return custom_labels.get(original_label, original_label)  # 매핑된 값이 없으면 원래 레이블 반환

# 라벨 이름을 저장할 리스트 초기화
detected_labels = []

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

for mask in masks:
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 라벨 이름 저장
    detected_labels.append(label)

    # 객체 경계 그리기 및 텍스트 레이블 추가
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), label, color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

plt.show()

# 모든 검출된 라벨 출력
print("Detected Labels:")
for i, label in enumerate(detected_labels, 1):
    print(f"{i}: {label}")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# 임시 클래스 매핑
custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    # 필요한 경우 다른 레이블도 매핑
}

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    # 마스크된 객체 영역만 잘라내기
    masked_image = Image.fromarray(image * mask[..., np.newaxis])

    # 전처리 후 모델에 입력
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    # 분류 결과 얻기
    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 커스텀 라벨로 매핑
    return custom_labels.get(original_label, original_label)  # 매핑된 값이 없으면 원래 레이블 반환

# 라벨 이름을 저장할 리스트 초기화
detected_labels = []

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

for i, mask in enumerate(masks, 1):
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 라벨 이름 저장
    detected_labels.append(label)

    # 객체 경계 그리기 및 텍스트 레이블 추가 (번호 포함)
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), f"{i}: {label}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

plt.show()

# 모든 검출된 라벨 출력
print("Detected Labels:")
for i, label in enumerate(detected_labels, 1):
    print(f"{i}: {label}")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# 임시 클래스 매핑 (오류 발생 시 교정)
custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    "cleaver": "industrial",
    "tennis ball": "industrial",
    "ping-pong ball": "industrial",
    # 필요한 경우 다른 레이블도 매핑
}

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    # 마스크된 객체 영역만 잘라내기
    masked_image = Image.fromarray(image * mask[..., np.newaxis])

    # 전처리 후 모델에 입력
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    # 분류 결과 얻기
    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 커스텀 라벨로 매핑
    corrected_label = custom_labels.get(original_label, original_label)  # 매핑된 값이 없으면 원래 레이블 반환

    # 신뢰도 검토 후 "Unknown"으로 설정 (선택사항)
    # if logit_value < some_threshold:
    #     corrected_label = "Unknown"

    return corrected_label

# 라벨 이름을 저장할 리스트 초기화
detected_labels = []

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

for i, mask in enumerate(masks, 1):
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 라벨 이름 저장
    detected_labels.append(label)

    # 객체 경계 그리기 및 텍스트 레이블 추가 (번호 포함)
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), f"{i}: {label}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

plt.show()

# 모든 검출된 라벨 출력
print("Detected Labels:")
for i, label in enumerate(detected_labels, 1):
    print(f"{i}: {label}")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# 임시 클래스 매핑 (오류 발생 시 교정)
custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    "cleaver": "industrial",
    "tennis ball": "industrial",
    "ping-pong ball": "industrial",
    "nematode": "building",         # 잘못된 레이블을 building으로 매핑
    "syringe": "building",          # 잘못된 레이블을 building으로 매핑
    "home theater": "building",     # 잘못된 레이블을 building으로 매핑
    # 필요한 경우 다른 레이블도 추가 매핑 가능
}

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    # 마스크된 객체 영역만 잘라내기
    masked_image = Image.fromarray(image * mask[..., np.newaxis])

    # 전처리 후 모델에 입력
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    # 분류 결과 얻기
    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 커스텀 라벨로 매핑
    corrected_label = custom_labels.get(original_label, original_label)  # 매핑된 값이 없으면 원래 레이블 반환

    return corrected_label

# 라벨 이름을 저장할 리스트 초기화
detected_labels = []

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

for i, mask in enumerate(masks, 1):
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 라벨 이름 저장
    detected_labels.append(label)

    # 객체 경계 그리기 및 텍스트 레이블 추가 (번호 포함)
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), f"{i}: {label}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

plt.show()

# 모든 검출된 라벨 출력
print("Detected Labels:")
for i, label in enumerate(detected_labels, 1):
    print(f"{i}: {label}")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json
import os

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# JSON 파일 경로
custom_labels_file = 'custom_labels.json'

# 초기 레이블링 규칙 (최초 실행 시 사용)
initial_custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    "cleaver": "industrial",
    "tennis ball": "industrial",
    "ping-pong ball": "industrial",
    "nematode": "building",         # 잘못된 레이블을 building으로 매핑
    "syringe": "building",          # 잘못된 레이블을 building으로 매핑
    "home theater": "building",     # 잘못된 레이블을 building으로 매핑
    # 필요한 경우 다른 레이블도 추가 매핑 가능
}

# JSON 파일로 저장하는 함수
def save_custom_labels(custom_labels, file_path):
    with open(file_path, 'w') as f:
        json.dump(custom_labels, f, indent=4)

# JSON 파일에서 불러오는 함수
def load_custom_labels(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        return {}

# 기존 규칙을 파일로 저장 (최초 실행 시)
if not os.path.exists(custom_labels_file):
    save_custom_labels(initial_custom_labels, custom_labels_file)

# 코드 실행 시 JSON 파일에서 레이블링 규칙 로드
custom_labels = load_custom_labels(custom_labels_file)

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    masked_image = Image.fromarray(image * mask[..., np.newaxis])
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 로드된 커스텀 레이블로 매핑
    corrected_label = custom_labels.get(original_label, original_label)

    return corrected_label

# 라벨 이름을 저장할 리스트 초기화
detected_labels = []

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

for i, mask in enumerate(masks, 1):
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 라벨 이름 저장
    detected_labels.append(label)

    # 객체 경계 그리기 및 텍스트 레이블 추가 (번호 포함)
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), f"{i}: {label}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

plt.show()

# 모든 검출된 라벨 출력
print("Detected Labels:")
for i, label in enumerate(detected_labels, 1):
    print(f"{i}: {label}")


Output hidden; open in https://colab.research.google.com to view.

In [15]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json
import os

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map3.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# JSON 파일 경로
custom_labels_file = 'custom_labels.json'

# 초기 레이블링 규칙 (최초 실행 시 사용)
initial_custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    "cleaver": "industrial",
    "tennis ball": "industrial",
    "ping-pong ball": "industrial",
    "nematode": "building",         # 잘못된 레이블을 building으로 매핑
    "syringe": "building",          # 잘못된 레이블을 building으로 매핑
    "home theater": "building",     # 잘못된 레이블을 building으로 매핑
    "match": "building",            # match를 building으로 매핑
    "microphone": "building",       # microphone을 building으로 매핑
}

# JSON 파일로 저장하는 함수
def save_custom_labels(custom_labels, file_path):
    with open(file_path, 'w') as f:
        json.dump(custom_labels, f, indent=4)

# JSON 파일에서 불러오는 함수
def load_custom_labels(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        return {}

# 기존 규칙을 파일로 저장 (최초 실행 시)
if not os.path.exists(custom_labels_file):
    save_custom_labels(initial_custom_labels, custom_labels_file)

# 코드 실행 시 JSON 파일에서 레이블링 규칙 로드
custom_labels = load_custom_labels(custom_labels_file)

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    masked_image = Image.fromarray(image * mask[..., np.newaxis])
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 로드된 커스텀 레이블로 매핑
    corrected_label = custom_labels.get(original_label, original_label)

    return corrected_label

# 라벨 이름을 저장할 리스트 초기화
detected_labels = []

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

for i, mask in enumerate(masks, 1):
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 라벨 이름 저장
    detected_labels.append(label)

    # 객체 경계 그리기 및 텍스트 레이블 추가 (번호 포함)
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), f"{i}: {label}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

plt.show()

# 모든 검출된 라벨 출력
print("Detected Labels:")
for i, label in enumerate(detected_labels, 1):
    print(f"{i}: {label}")


Output hidden; open in https://colab.research.google.com to view.

In [16]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json
import os
from shapely.geometry import Polygon, Point, LineString

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map3.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# JSON 파일 경로
custom_labels_file = 'custom_labels.json'

# 초기 레이블링 규칙 (최초 실행 시 사용)
initial_custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    "cleaver": "industrial",
    "tennis ball": "industrial",
    "ping-pong ball": "industrial",
    "nematode": "building",         # 잘못된 레이블을 building으로 매핑
    "syringe": "building",          # 잘못된 레이블을 building으로 매핑
    "home theater": "building",     # 잘못된 레이블을 building으로 매핑
    "match": "building",            # match를 building으로 매핑
    "microphone": "building",       # microphone을 building으로 매핑
}

# JSON 파일로 저장하는 함수
def save_custom_labels(custom_labels, file_path):
    with open(file_path, 'w') as f:
        json.dump(custom_labels, f, indent=4)

# JSON 파일에서 불러오는 함수
def load_custom_labels(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        return {}

# 기존 규칙을 파일로 저장 (최초 실행 시)
if not os.path.exists(custom_labels_file):
    save_custom_labels(initial_custom_labels, custom_labels_file)

# 코드 실행 시 JSON 파일에서 레이블링 규칙 로드
custom_labels = load_custom_labels(custom_labels_file)

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    masked_image = Image.fromarray(image * mask[..., np.newaxis])
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 로드된 커스텀 레이블로 매핑
    corrected_label = custom_labels.get(original_label, original_label)

    return corrected_label

# 라벨 이름을 저장할 리스트 초기화
detected_labels = []

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

# 장애물 다각형을 저장할 리스트 초기화
obstacle_polygons = []

for i, mask in enumerate(masks, 1):
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 라벨 이름 저장
    detected_labels.append(label)

    # 객체 경계 그리기 및 텍스트 레이블 추가 (번호 포함)
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), f"{i}: {label}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

    # 장애물 다각형 추가
    obstacle_polygons.append(Polygon(zip(x, y)))

# RRT 알고리즘 파라미터 설정
max_iter = 1000
step_size = 10

# 시작점과 도착점 임의 지정 (이미지 크기에 따라 적절히 조정)
start = np.array([100, 100])
goal = np.array([400, 400])

def is_collision_free(p1, p2, obstacles):
    """p1과 p2를 연결하는 선이 장애물과 충돌하는지 확인"""
    line = LineString([p1, p2])
    return all(not line.intersects(obstacle) for obstacle in obstacles)

def rrt(start, goal, obstacles, max_iter=1000, step_size=10):
    """RRT 알고리즘 구현"""
    nodes = [start]
    for _ in range(max_iter):
        # 무작위 샘플링
        rand_point = np.random.rand(2) * np.array(image_np.shape[1::-1])
        # 가장 가까운 노드 찾기
        nearest_node = min(nodes, key=lambda n: np.linalg.norm(n - rand_point))
        direction = (rand_point - nearest_node) / np.linalg.norm(rand_point - nearest_node)
        new_node = nearest_node + direction * step_size
        new_node = np.clip(new_node, 0, np.array(image_np.shape[1::-1]) - 1)

        if is_collision_free(nearest_node, new_node, obstacles):
            nodes.append(new_node)
            # 목표 지점 근처에 도달하면 종료
            if np.linalg.norm(new_node - goal) < step_size:
                nodes.append(goal)
                break

    # 경로 생성
    path = [goal]
    while np.linalg.norm(path[-1] - start) > step_size:
        nearest_node = min(nodes, key=lambda n: np.linalg.norm(n - path[-1]))
        path.append(nearest_node)
    path.append(start)
    return path[::-1]

# RRT로 경로 찾기
path = rrt(start, goal, obstacle_polygons, max_iter=max_iter, step_size=step_size)

# 경로 시각화
plt.plot([p[0] for p in path], [p[1] for p in path], '-o', color='blue', markersize=5)
plt.scatter(*start, color='green', s=100, label='Start')
plt.scatter(*goal, color='red', s=100, label='Goal')

# 장애물 시각화
for obstacle in obstacle_polygons:
    x, y = obstacle.exterior.xy
    plt.fill(x, y, color='gray', alpha=0.5)

plt.axis('off')
plt.legend()
plt.show()

# 모든 검출된 라벨 출력
print("Detected Labels:")
for i, label in enumerate(detected_labels, 1):
    print(f"{i}: {label}")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json
import os
from shapely.geometry import Polygon, Point, LineString

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map3.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# JSON 파일 경로
custom_labels_file = 'custom_labels.json'

# 초기 레이블링 규칙 (최초 실행 시 사용)
initial_custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    "cleaver": "industrial",
    "tennis ball": "industrial",
    "ping-pong ball": "industrial",
    "nematode": "building",         # 잘못된 레이블을 building으로 매핑
    "syringe": "building",          # 잘못된 레이블을 building으로 매핑
    "home theater": "building",     # 잘못된 레이블을 building으로 매핑
    "match": "building",            # match를 building으로 매핑
    "microphone": "building",       # microphone을 building으로 매핑
}

# JSON 파일로 저장하는 함수
def save_custom_labels(custom_labels, file_path):
    with open(file_path, 'w') as f:
        json.dump(custom_labels, f, indent=4)

# JSON 파일에서 불러오는 함수
def load_custom_labels(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        return {}

# 기존 규칙을 파일로 저장 (최초 실행 시)
if not os.path.exists(custom_labels_file):
    save_custom_labels(initial_custom_labels, custom_labels_file)

# 코드 실행 시 JSON 파일에서 레이블링 규칙 로드
custom_labels = load_custom_labels(custom_labels_file)

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    masked_image = Image.fromarray(image * mask[..., np.newaxis])
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 로드된 커스텀 레이블로 매핑
    corrected_label = custom_labels.get(original_label, original_label)

    return corrected_label

# 라벨 이름을 저장할 리스트 초기화
detected_labels = []

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

# 장애물 다각형을 저장할 리스트 초기화
obstacle_polygons = []

for i, mask in enumerate(masks, 1):
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 라벨 이름 저장
    detected_labels.append(label)

    # 객체 경계 그리기 및 텍스트 레이블 추가 (번호 포함)
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), f"{i}: {label}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

    # 장애물 다각형 추가
    obstacle_polygons.append(Polygon(zip(x, y)))

# RRT 알고리즘 파라미터 설정
max_iter = 1000
step_size = 10

# 임의의 시작점과 도착점을 10개 생성 (이미지 크기에 따라 적절히 조정)
num_paths = 10
start_goal_pairs = []
for _ in range(num_paths):
    start = np.random.randint(0, image_np.shape[1], 2)
    goal = np.random.randint(0, image_np.shape[1], 2)
    start_goal_pairs.append((start, goal))

def is_collision_free(p1, p2, obstacles):
    """p1과 p2를 연결하는 선이 장애물과 충돌하는지 확인"""
    line = LineString([p1, p2])
    return all(not line.intersects(obstacle) for obstacle in obstacles)

def rrt(start, goal, obstacles, max_iter=1000, step_size=10):
    """RRT 알고리즘 구현"""
    nodes = [start]
    for _ in range(max_iter):
        # 무작위 샘플링
        rand_point = np.random.rand(2) * np.array(image_np.shape[1::-1])
        # 가장 가까운 노드 찾기
        nearest_node = min(nodes, key=lambda n: np.linalg.norm(n - rand_point))
        direction = (rand_point - nearest_node) / np.linalg.norm(rand_point - nearest_node)
        new_node = nearest_node + direction * step_size
        new_node = np.clip(new_node, 0, np.array(image_np.shape[1::-1]) - 1)

        if is_collision_free(nearest_node, new_node, obstacles):
            nodes.append(new_node)
            # 목표 지점 근처에 도달하면 종료
            if np.linalg.norm(new_node - goal) < step_size:
                nodes.append(goal)
                break

    # 경로 생성
    path = [goal]
    while np.linalg.norm(path[-1] - start) > step_size:
        nearest_node = min(nodes, key=lambda n: np.linalg.norm(n - path[-1]))
        path.append(nearest_node)
    path.append(start)
    return path[::-1]

# 모든 경로 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

for start, goal in start_goal_pairs:
    # RRT로 경로 찾기
    path = rrt(start, goal, obstacle_polygons, max_iter=max_iter, step_size=step_size)

    # 경로 시각화
    plt.plot([p[0] for p in path], [p[1] for p in path], '-o', markersize=5)
    plt.scatter(*start, color='green', s=100, label='Start')
    plt.scatter(*goal, color='red', s=100, label='Goal')

# 장애물 시각화
for obstacle in obstacle_polygons:
    x, y = obstacle.exterior.xy
    plt.fill(x, y, color='gray', alpha=0.5)

plt.legend()
plt.show()

# 모든 검출된 라벨 출력
print("Detected Labels:")
for i, label in enumerate(detected_labels, 1):
    print(f"{i}: {label}")


In [17]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json
import random

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map3.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# 임시 클래스 매핑
custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    # 필요한 경우 다른 레이블도 매핑
}

# 랜덤 색상 생성 함수
def random_color():
    return [random.randint(0, 255) for _ in range(3)]

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    # 마스크된 객체 영역만 잘라내기
    masked_image = Image.fromarray(image * mask[..., np.newaxis])

    # 전처리 후 모델에 입력
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    # 분류 결과 얻기
    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 커스텀 라벨로 매핑
    return custom_labels.get(original_label, original_label)  # 매핑된 값이 없으면 원래 레이블 반환

# 이미지 초기화
output_image = image_np.copy()

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))

for mask in masks:
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 객체 경계 그리기 및 텍스트 레이블 추가
    y, x = np.where(mask_np)
    color = random_color()  # 랜덤 색상 선택
    output_image[mask_np] = color  # 객체 영역에 색상 적용

    plt.text(x.min(), y.min(), label, color='white', fontsize=12, bbox=dict(facecolor='black', alpha=0.6))

# 이미지 출력
plt.imshow(output_image)
plt.axis('off')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/naver_map.jpg'

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import requests
import json
import os
from shapely.geometry import Polygon, Point, LineString

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# SAM 모델 설정
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# 이미지 로드
image = Image.open("/content/sample_data/naver_map3.jpg").convert("RGB")
image_np = np.array(image)

# 마스크 생성
masks = mask_generator.generate(image_np)
print(f"Generated {len(masks)} masks.")
print(masks[0].keys())

# 객체 분류를 위한 이미지 분류 모델 로드 (여기서는 ResNet 사용)
classifier = models.resnet50(pretrained=True)
classifier.eval()
classifier.to(device)

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 클래스 레이블 로드
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = requests.get(LABELS_URL)
imagenet_labels = json.loads(response.text)

# JSON 파일 경로
custom_labels_file = 'custom_labels.json'

# 초기 레이블링 규칙 (최초 실행 시 사용)
initial_custom_labels = {
    "spotlight": "building",
    "street_sign": "road",
    "parking_meter": "car",
    "cleaver": "industrial",
    "tennis ball": "industrial",
    "ping-pong ball": "industrial",
    "nematode": "building",         # 잘못된 레이블을 building으로 매핑
    "syringe": "building",          # 잘못된 레이블을 building으로 매핑
    "home theater": "building",     # 잘못된 레이블을 building으로 매핑
    "match": "building",            # match를 building으로 매핑
    "microphone": "building",       # microphone을 building으로 매핑
}

# JSON 파일로 저장하는 함수
def save_custom_labels(custom_labels, file_path):
    with open(file_path, 'w') as f:
        json.dump(custom_labels, f, indent=4)

# JSON 파일에서 불러오는 함수
def load_custom_labels(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        return {}

# 기존 규칙을 파일로 저장 (최초 실행 시)
if not os.path.exists(custom_labels_file):
    save_custom_labels(initial_custom_labels, custom_labels_file)

# 코드 실행 시 JSON 파일에서 레이블링 규칙 로드
custom_labels = load_custom_labels(custom_labels_file)

# 객체 분리 및 분류
def classify_masked_region(image, mask):
    masked_image = Image.fromarray(image * mask[..., np.newaxis])
    input_tensor = preprocess(masked_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = classifier(input_tensor)

    _, predicted_class = output.max(1)
    original_label = imagenet_labels[predicted_class.item()]

    # 로드된 커스텀 레이블로 매핑
    corrected_label = custom_labels.get(original_label, original_label)

    return corrected_label

# 라벨 이름을 저장할 리스트 초기화
detected_labels = []

# 객체 분리 및 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

# 장애물 다각형을 저장할 리스트 초기화
obstacle_polygons = []

for i, mask in enumerate(masks, 1):
    mask_np = mask['segmentation']

    # 마스크된 객체를 분류
    label = classify_masked_region(image_np, mask_np)

    # 라벨 이름 저장
    detected_labels.append(label)

    # 객체 경계 그리기 및 텍스트 레이블 추가 (번호 포함)
    y, x = np.where(mask_np)
    plt.text(x.min(), y.min(), f"{i}: {label}", color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.contour(mask_np, colors='red', linewidths=1)

    # 장애물 다각형 추가
    obstacle_polygons.append(Polygon(zip(x, y)))

# RRT 알고리즘 파라미터 설정
max_iter = 1000
step_size = 10

# 임의의 시작점과 도착점을 10개 생성 (이미지 크기에 따라 적절히 조정)
num_paths = 10
start_goal_pairs = []
for _ in range(num_paths):
    start = np.random.randint(0, image_np.shape[1], 2)
    goal = np.random.randint(0, image_np.shape[1], 2)
    start_goal_pairs.append((start, goal))

def is_collision_free(p1, p2, obstacles):
    """p1과 p2를 연결하는 선이 장애물과 충돌하는지 확인"""
    line = LineString([p1, p2])
    return all(not line.intersects(obstacle) for obstacle in obstacles)

def rrt(start, goal, obstacles, max_iter=1000, step_size=10):
    """RRT 알고리즘 구현"""
    nodes = [start]
    for _ in range(max_iter):
        # 무작위 샘플링
        rand_point = np.random.rand(2) * np.array(image_np.shape[1::-1])
        # 가장 가까운 노드 찾기
        nearest_node = min(nodes, key=lambda n: np.linalg.norm(n - rand_point))
        direction = (rand_point - nearest_node) / np.linalg.norm(rand_point - nearest_node)
        new_node = nearest_node + direction * step_size
        new_node = np.clip(new_node, 0, np.array(image_np.shape[1::-1]) - 1)

        if is_collision_free(nearest_node, new_node, obstacles):
            nodes.append(new_node)
            # 목표 지점 근처에 도달하면 종료
            if np.linalg.norm(new_node - goal) < step_size:
                nodes.append(goal)
                break

    # 경로 생성
    path = [goal]
    while np.linalg.norm(path[-1] - start) > step_size:
        nearest_node = min(nodes, key=lambda n: np.linalg.norm(n - path[-1]))
        path.append(nearest_node)
    path.append(start)
    return path[::-1]

# 모든 경로 시각화
plt.figure(figsize=(20,20))
plt.imshow(image_np)
plt.axis('off')

for start, goal in start_goal_pairs:
    # RRT로 경로 찾기
    path = rrt(start, goal, obstacle_polygons, max_iter=max_iter, step_size=step_size)

    # 경로 시각화
    plt.plot([p[0] for p in path], [p[1] for p in path], '-o', markersize=5)
    plt.scatter(*start, color='green', s=100, label='Start')
    plt.scatter(*goal, color='red', s=100, label='Goal')

# 장애물 시각화
for obstacle in obstacle_polygons:
    x, y = obstacle.exterior.xy
    plt.fill(x, y, color='gray', alpha=0.5)

plt.legend()
plt.show()

# 모든 검출된 라벨 출력
print("Detected Labels:")
for i, label in enumerate(detected_labels, 1):
    print(f"{i}: {label}")


/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Generated 111 masks.
dict_keys(['segmentation', 'area', 'bbox', 'predicted_iou', 'point_coords', 'stability_score', 'crop_box'])


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [19]:
! ps -ef | grep python

root          57       7  0 02:17 ?        00:00:06 [python3] <defunct>
root          58       7  0 02:17 ?        00:00:00 python3 /usr/local/bin/colab-fileshim.py
root         107       7  0 02:17 ?        00:00:05 /usr/bin/python3 /usr/local/bin/jupyter-notebook
root        2372     107 11 02:27 ?        00:06:26 /usr/bin/python3 -m colab_kernel_launcher -f /ro
root        2421       1  0 02:27 ?        00:00:07 /usr/bin/python3 /usr/local/lib/python3.10/dist-
root       16000    2372  0 03:21 ?        00:00:00 /bin/bash -c  ps -ef | grep python
root       16002   16000  0 03:21 ?        00:00:00 grep python


In [ ]:
! kill -9 {107}

## Automatic mask generation options

There are several tunable parameters in automatic mask generation that control how densely points are sampled and what the thresholds are for removing low quality or duplicate masks. Additionally, generation can be automatically run on crops of the image to get improved performance on smaller objects, and post-processing can remove stray pixels and holes. Here is an example configuration that samples more masks:

In [ ]:
mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

In [ ]:
masks2 = mask_generator_2.generate(image)

In [ ]:
len(masks2)

679

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks2)
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))

# 흑백으로 이미지를 표시하기 위해 cmap='gray'를 추가합니다.
plt.imshow(image, cmap='gray')
show_anns(masks2)
plt.axis('off')
plt.show()